In [1]:
#聲音處理套件
import librosa
import librosa.display
#資料處理套件
import pandas as pd
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [7]:
#音訊存放位置
ESC_DATA_PATH = "ESC/Fold"

def getTrainDataset_val135(fold, soundLength, labelnum):
    x_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_" + str(labelnum) + ".pt"
    x_val1_path = ESC_DATA_PATH + str(fold) + "/" + str(1) + "s/x_val.pt"
    y_val1_path = ESC_DATA_PATH + str(fold) + "/" + str(1) + "s/y_val_" + str(labelnum) + ".pt"
    x_val3_path = ESC_DATA_PATH + str(fold) + "/" + str(3) + "s/x_val.pt"
    y_val3_path = ESC_DATA_PATH + str(fold) + "/" + str(3) + "s/y_val_" + str(labelnum) + ".pt"
    x_val5_path = ESC_DATA_PATH + str(fold) + "/" + str(5) + "s/x_val.pt"
    y_val5_path = ESC_DATA_PATH + str(fold) + "/" + str(5) + "s/y_val_" + str(labelnum) + ".pt"
    
    x_train = torch.load(x_train_path)
    y_train = torch.load(y_train_path)
    x_val1 = torch.load(x_val1_path)
    y_val1 = torch.load(y_val1_path)
    x_val3 = torch.load(x_val3_path)
    y_val3 = torch.load(y_val3_path)
    x_val5 = torch.load(x_val5_path)
    y_val5 = torch.load(y_val5_path)
    
    #打包
    #train = torch.utils.data.TensorDataset(x_train,y_train)
    #val1 = torch.utils.data.TensorDataset(x_val1,y_val1)
    #val3 = torch.utils.data.TensorDataset(x_val3,y_val3)
    #val5 = torch.utils.data.TensorDataset(x_val5,y_val5)
    return x_val1, y_val1

In [8]:
#資料抓入(ESC10_Train,ESC10_Test 1s/3s/5s)
x_train, y_train = getTrainDataset_val135(1, 1, 10)
print(x_train.shape)

torch.Size([400, 1, 128, 87])


In [5]:
#1D音訊存放位置
ESC_DATA_PATH_1D = "ESC_1D/Fold"
def getTrainDataset_val135_1D(fold, soundLength, labelnum):
    x_train_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(soundLength) + "s/y_train_" + str(labelnum) + ".pt"
    x_val1_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(1) + "s/x_val.pt"
    y_val1_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(1) + "s/y_val_" + str(labelnum) + ".pt"
    x_val3_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(3) + "s/x_val.pt"
    y_val3_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(3) + "s/y_val_" + str(labelnum) + ".pt"
    x_val5_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(5) + "s/x_val.pt"
    y_val5_path = ESC_DATA_PATH_1D + str(fold) + "/" + str(5) + "s/y_val_" + str(labelnum) + ".pt"
    
    x_train = torch.load(x_train_path)
    y_train = torch.load(y_train_path)
    x_val1 = torch.load(x_val1_path)
    y_val1 = torch.load(y_val1_path)
    x_val3 = torch.load(x_val3_path)
    y_val3 = torch.load(y_val3_path)
    x_val5 = torch.load(x_val5_path)
    y_val5 = torch.load(y_val5_path)
    
    #打包
    #train = torch.utils.data.TensorDataset(x_train,y_train)
    #val1 = torch.utils.data.TensorDataset(x_val1,y_val1)
    #val3 = torch.utils.data.TensorDataset(x_val3,y_val3)
    #val5 = torch.utils.data.TensorDataset(x_val5,y_val5)
    
    return x_val1, y_val1

In [6]:
#資料抓入(ESC10_Train,ESC10_Test 1s/3s/5s)
x_train, y_train = getTrainDataset_val135_1D(1, 1, 10)
print(x_train.shape)


torch.Size([400, 1, 44100])


In [2]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [3]:
#音訊存放位置
ESC_DATA_PATH = "C:/Users/Jia-Wei Chang/hu_file/ESC20220524_pytorch/Preprocessing/ESC/Fold"
C_DATA_PATH = "C:/Users/Jia-Wei Chang/hu_file/ESC20220524_pytorch/Preprocessing/Chainsaw/"

def getTrainDataset(fold, soundLength, labelnum):
    x_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_" + str(labelnum) + ".pt"
    x_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_val.pt"
    y_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val_" + str(labelnum) + ".pt"
    
    x_train = torch.load(x_train_path)
    y_train = torch.load(y_train_path)
    x_val = torch.load(x_val_path)
    y_val = torch.load(y_val_path)
    
    #打包
    train = torch.utils.data.TensorDataset(x_train,y_train)
    val = torch.utils.data.TensorDataset(x_val,y_val)
    
    return train, val

def getTestDataset(soundLength):
    x_test_path = C_DATA_PATH + str(soundLength) + "s/x_test.pt"
    y_test_path = C_DATA_PATH + str(soundLength) + "s/y_test.pt"
    
    x_test = torch.load(x_test_path)
    y_test = torch.load(y_test_path)
    
    #打包
    test = torch.utils.data.TensorDataset(x_test,y_test)    
    
    return test

In [4]:
#測試模型對於電鋸資料集的準確度
def test_model(test_loader, input_shape):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(input_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集
        test_accuracy_array.append(test_model(test_loader, input_shape))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test_accuracy_array

In [7]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test_accuracy_array = []

#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = 5
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = (-1,1,128,431)
#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ",fold)
    print("----------------------------------------------------------------------------------------------------------------")
    #建立模型
    model = LeNet(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    #loss方法
    loss_func = nn.CrossEntropyLoss()

    #資料預處理
    train, val = getTrainDataset(fold, sound, num_class)
    test_loader = getTestDataset(sound)

    #分批方法
    train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
    val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
    
    #訓練模型
    train_loss, train_accuracy, val_loss, val_accuracy, test_accuracy = train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader)
    
    #將gpu資料取回
    train_loss = torch.stack(train_loss,dim=-1).cpu()
    train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    val_loss = torch.stack(val_loss,dim=-1).cpu()
    val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
    for i in range(num_epochs):
        # 寫入資料
        f.write("fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test_accuracy: {:.6f}%\n".format(fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    
    train_loss_array.append(train_loss)
    train_accuracy_array.append(train_accuracy)
    val_loss_array.append(val_loss)
    val_accuracy_array.append(val_accuracy)
    test_accuracy_array.append(test_accuracy)

f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
Train Epoch: 1/30 Train_Loss: 2.332378387451172 Train_acc: 6.250000% Val_Loss: 2.1358704566955566 Val_accuracy: 10.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 2.206308364868164 Train_acc: 10.312500% Val_Loss: 2.1493890285491943 Val_accuracy: 10.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 2.1431164741516113 Train_acc: 11.875000% Val_Loss: 2.1979455947875977 Val_accuracy: 18.750000%
Test accuracy of the model: 0.000000%
Train Epoch: 4/30 Train_Loss: 2.1362226009368896 Train_acc: 14.062500% Val_Loss: 2.150585651397705 Val_accuracy: 10.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 5/30 Train_Loss: 2.1194586753845215 Train_acc: 11.250000% Val_Loss: 2.1916396617889404 Val_accuracy: 12.500000%
Test accuracy of the model: 0.000000%
Train Epoch: 6/30 Train_Loss: 2.0950775146484375 Train_acc: 14.3750

Train Epoch: 20/30 Train_Loss: 1.8596094846725464 Train_acc: 37.187500% Val_Loss: 1.7387737035751343 Val_accuracy: 37.500000%
Test accuracy of the model: 87.000000%
Train Epoch: 21/30 Train_Loss: 1.852616310119629 Train_acc: 35.937500% Val_Loss: 1.7841469049453735 Val_accuracy: 35.000000%
Test accuracy of the model: 87.000000%
Train Epoch: 22/30 Train_Loss: 1.8926260471343994 Train_acc: 31.875000% Val_Loss: 1.863136887550354 Val_accuracy: 31.250000%
Test accuracy of the model: 87.000000%
Train Epoch: 23/30 Train_Loss: 1.8387426137924194 Train_acc: 30.625000% Val_Loss: 1.7916203737258911 Val_accuracy: 37.500000%
Test accuracy of the model: 89.000000%
Train Epoch: 24/30 Train_Loss: 1.7522733211517334 Train_acc: 32.187500% Val_Loss: 1.822809100151062 Val_accuracy: 38.750000%
Test accuracy of the model: 87.000000%
Train Epoch: 25/30 Train_Loss: 1.9333364963531494 Train_acc: 34.062500% Val_Loss: 1.8983486890792847 Val_accuracy: 38.750000%
Test accuracy of the model: 86.000000%
Train Epoch: 

Train Epoch: 9/30 Train_Loss: 2.039085865020752 Train_acc: 24.062500% Val_Loss: 2.0254690647125244 Val_accuracy: 23.750000%
Test accuracy of the model: 26.000000%
Train Epoch: 10/30 Train_Loss: 1.9811614751815796 Train_acc: 26.250000% Val_Loss: 2.140007257461548 Val_accuracy: 32.500000%
Test accuracy of the model: 43.000000%
Train Epoch: 11/30 Train_Loss: 1.9328829050064087 Train_acc: 27.812500% Val_Loss: 1.8953322172164917 Val_accuracy: 30.000000%
Test accuracy of the model: 56.000000%
Train Epoch: 12/30 Train_Loss: 1.9619418382644653 Train_acc: 28.750000% Val_Loss: 1.9239606857299805 Val_accuracy: 30.000000%
Test accuracy of the model: 57.000000%
Train Epoch: 13/30 Train_Loss: 1.8724558353424072 Train_acc: 29.687500% Val_Loss: 2.080151081085205 Val_accuracy: 30.000000%
Test accuracy of the model: 58.000000%
Train Epoch: 14/30 Train_Loss: 1.8606175184249878 Train_acc: 30.000000% Val_Loss: 1.8502545356750488 Val_accuracy: 32.500000%
Test accuracy of the model: 63.000000%
Train Epoch: 1

Train Epoch: 29/30 Train_Loss: 1.7137764692306519 Train_acc: 34.687500% Val_Loss: 1.7730510234832764 Val_accuracy: 33.750000%
Test accuracy of the model: 47.000000%
Train Epoch: 30/30 Train_Loss: 1.6904025077819824 Train_acc: 34.687500% Val_Loss: 1.7796146869659424 Val_accuracy: 33.750000%
Test accuracy of the model: 48.000000%
